# Capstone - 2016 Immigration and Temperature Data 

### Data Engineering Capstone Project

#### Project Summary
The goal of this project is to create an ETL pipeline using I94 immigration data and city temperature data to form a database that is optimized for queries on immigration events. This database can be used to answer questions relating immigration behavior to destination temperature e.g., do people tend to immigrate to warmer places?

The project follows the follow steps:
* Step 1: Scope the Project and Gather Data
* Step 2: Explore and Assess the Data
* Step 3: Define the Data Model
* Step 4: Run ETL to Model the Data
* Step 5: Complete Project Write Up

In [2]:
#Used below libraries
import os
from datetime import datetime,timedelta
import pandas as pd, re
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf
from pyspark.sql import types as T
from concurrent.futures import ThreadPoolExecutor
import concurrent.futures
import json

### Step 1: Scope the Project and Gather Data

#### Scope 
In this project, we will aggregate I94 immigration data by destination city to form our first dimension table. Next we will aggregate city temperature data by city to form the second dimension table. The two datasets will be joined on destination city to form the fact table. The final database is optimized to query on immigration events to determine if temperature affects the selection of destination cities. Spark will be used to process the data.

#### Describe and Gather Data 
Describe the data sets you're using. Where did it come from? What type of information is included? 

The [I94 immigration data](https://travel.trade.gov/research/reports/i94/historical/2016.html) comes from the US National Tourism and Trade Office. It is provided in SAS7BDAT format which is a binary database storage format. Please refer [I94_SAS_Labels_Descriptions.SAS]('I94_SAS_Labels_Descriptions.SAS') document for full reference. Some relevant attributes include:

##### Source Type: __SAS Generated Data__ and __Static Text Files__

* i94yr = 4 digit year
* i94mon = numeric month
* i94cit = 3 digit code of origin city
* i94port = 3 character code of destination USA city
* arrdate = arrival date in the USA
* i94mode = 1 digit travel code
* depdate = departure date from the USA
* i94visa = reason for immigration

The [temperature data](https://www.kaggle.com/berkeleyearth/climate-change-earth-surface-temperature-data) comes from Kaggle. It is provided in csv format. Some relevant attributes include:

##### Source Type: __CSV Data__

* Dt = date
* AverageTemperature = average temperature
* City = city name
* Country = country name
* Latitude= latitude
* Longitude = longitude

The [U.S. City Demographic Data](https://public.opendatasoft.com/explore/dataset/us-cities-demographics/export/) data comes from OpenSoft.

##### Source Type: __JSON Data__

* Average_Household_Size
* City
* Count
* Female_Population
* Foreign_Born
* Male_Population
* Median_Age
* Number_of_Veterans
* Race
* State_Code
* Total_Population
* Record_Timestamp

The [Airport Code Table](https://datahub.io/core/airport-codes#data) is a simple table of airport codes and corresponding cities.

##### Source Type: __CSV Data__

* Ident
* Type 
* Name 
* Elevation_Ft 
* Continent 
* Iso_Country
* Iso_Region 
* Municipality 
* Gps_Code 
* Iata_Code 
* Local_Code
* Coordinates


### Step 2: Explore and Assess the Data
#### Explore the Data 
* Read all sources into dataframes and explored the data
* Fixed the Data Type issues and found the PK columns of the data set

##### Immigration Data Analysis:


1. Based on Sample data, assuming the columns will have 100% of data if below the count is 1000 else less than 100%
* cicid     : 1000 Rows
* i94yr     : 1000 Rows
* i94mon    : 1000 Rows
* i94cit    : 1000 Rows
* i94res    : 1000 Rows
* i94port   : 1000 Rows
* arrdate   : 1000 Rows
* i94mode   : 1000 Rows
* i94addr   :  941 Rows
* depdate   :  951 Rows
* i94bir    : 1000 Rows
* i94visa   : 1000 Rows
* count     : 1000 Rows
* dtadfile  : 1000 Rows
* visapost  :  382 Rows
* occup     :    4 Rows
* entdepa   : 1000 Rows
* entdepd   :  954 Rows
* entdepu   :    0 Rows
* matflag   :  954 Rows
* biryear   : 1000 Rows
* dtaddto   : 1000 Rows
* gender    :  859 Rows
* insnum    :   35 Rows
* airline   :  967 Rows
* admnum    : 1000 Rows
* fltno     :  992 Rows
* visatype  : 1000 Rows

After analying the file, there are no duplicate records, so keeping all data

**Note:** Other files data is stright forward, please see the sample data and analysis in Jupyter cells.

#### Cleaning Steps
* Converted the data types to the right format and dropped the duplicatesif there are any duplicates. Please refer steps below

##### Immigration Data:

1. Converting all non date fields to human readable date format and passing default date as 01/01/1900.
2. Convert float data columns trypes to int.
3. Remove * infront of Airline code.
4. When reading other Text static files into dataframe, clean the signle quotes.

##### US Demographic Data:

1. Converting date in string format to date format.
2. Convert float data columns trypes to int.
3. Extract __fields__ dictionary into notmal fields and remove fields prefix from those.
4. Extract City from **iso_region** 


In [23]:
# Read immigration sample data
df_immigration_pd = pd.read_csv('immigration_data_sample.csv',header=0)
df_immigration_pd = df_immigration_pd.iloc[:,1:]

In [5]:
# Because the immigration data has 28 columns
pd.set_option('display.max_columns', 28)

In [24]:
# Display sample data
df_immigration_pd.head(5)

,cicid,i94yr,i94mon,i94cit,i94res,i94port,arrdate,i94mode,i94addr,depdate,i94bir,i94visa,count,dtadfile,visapost,occup,entdepa,entdepd,entdepu,matflag,biryear,dtaddto,gender,insnum,airline,admnum,fltno,visatype
0,4084316.0,2016.0,4.0,209.0,209.0,HHW,20566.0,1.0,HI,20573.0,61.0,2.0,1.0,20160422,NaN,NaN,G,O,NaN,M,1955.0,07202016,F,NaN,JL,5.658267e+10,00782,WT
1,4422636.0,2016.0,4.0,582.0,582.0,MCA,20567.0,1.0,TX,20568.0,26.0,2.0,1.0,20160423,MTR,NaN,G,R,NaN,M,1990.0,10222016,M,NaN,*GA,9.436200e+10,XBLNG,B2
2,1195600.0,2016.0,4.0,148.0,112.0,OGG,20551.0,1.0,FL,20571.0,76.0,2.0,1.0,20160407,NaN,NaN,G,O,NaN,M,1940.0,07052016,M,NaN,LH,5.578047e+10,00464,WT
3,5291768.0,2016.0,4.0,297.0,297.0,LOS,20572.0,1.0,CA,20581.0,25.0,2.0,1.0,20160428,DOH,NaN,G,O,NaN,M,1991.0,10272016,M,NaN,QR,9.478970e+10,00739,B2
4,985523.0,2016.0,4.0,111.0,111.0,CHM,20550.0,3.0,NY,20553.0,19.0,2.0,1.0,20160406,NaN,NaN,Z,K,NaN,M,1997.0,07042016,F,NaN,NaN,4.232257e+10,LAND,WT


In [25]:
#Diplay sample data flights from 209
df_immigration_pd.query('i94cit==209.0').head(5)

,cicid,i94yr,i94mon,i94cit,i94res,i94port,arrdate,i94mode,i94addr,depdate,i94bir,i94visa,count,dtadfile,visapost,occup,entdepa,entdepd,entdepu,matflag,biryear,dtaddto,gender,insnum,airline,admnum,fltno,visatype
0,4084316.0,2016.0,4.0,209.0,209.0,HHW,20566.0,1.0,HI,20573.0,61.0,2.0,1.0,20160422,NaN,NaN,G,O,NaN,M,1955.0,07202016,F,NaN,JL,5.658267e+10,00782,WT
11,5056736.0,2016.0,4.0,209.0,209.0,PHI,20571.0,1.0,HI,20575.0,72.0,2.0,1.0,20160427,NaN,NaN,G,O,NaN,M,1944.0,07252016,M,NaN,DL,5.933662e+10,00598,WT
24,2721962.0,2016.0,4.0,209.0,209.0,NEW,20559.0,1.0,HI,20562.0,41.0,2.0,1.0,20160415,NaN,NaN,O,O,NaN,M,1975.0,07132016,NaN,NaN,HA,5.621703e+10,00458,WT
42,5472659.0,2016.0,4.0,209.0,209.0,NEW,20573.0,1.0,NY,20579.0,8.0,2.0,1.0,20160429,NaN,NaN,G,O,NaN,M,2008.0,07272016,M,NaN,UA,5.947873e+10,00078,WT
46,861557.0,2016.0,4.0,209.0,209.0,SDP,20549.0,1.0,NaN,20552.0,46.0,2.0,1.0,20160405,NaN,NaN,G,I,NaN,M,1970.0,07032016,M,NaN,JL,5.566323e+10,00066,WT


In [26]:
#Verify Data types
df_immigration_pd.dtypes

cicid       float64
i94yr       float64
i94mon      float64
i94cit      float64
i94res      float64
i94port      object
arrdate     float64
i94mode     float64
i94addr      object
depdate     float64
i94bir      float64
i94visa     float64
count       float64
dtadfile      int64
visapost     object
occup        object
entdepa      object
entdepd      object
entdepu     float64
matflag      object
biryear     float64
dtaddto      object
gender       object
insnum      float64
airline      object
admnum      float64
fltno        object
visatype     object
dtype: object

In [27]:
# Gives Count of non-NA cells for each column
df_immigration_pd.count()

cicid       1000
i94yr       1000
i94mon      1000
i94cit      1000
i94res      1000
i94port     1000
arrdate     1000
i94mode     1000
i94addr      941
depdate      951
i94bir      1000
i94visa     1000
count       1000
dtadfile    1000
visapost     382
occup          4
entdepa     1000
entdepd      954
entdepu        0
matflag      954
biryear     1000
dtaddto     1000
gender       859
insnum        35
airline      967
admnum      1000
fltno        992
visatype    1000
dtype: int64

In [28]:
#Finding the PK column
len(df_immigration_pd.admnum.unique())

1000

In [36]:
# Create arrays for each field in i94port codes
i94port_cd = []
i94port_state = []
i94port_city = []
with open('immigration_i94port_valid.txt') as f:
     for line in f:
            line = line.rstrip().lstrip()
            if line:
                #print(line)
                try:
                    key = line.split('=')[0].rstrip().lstrip()[1:-1]
                    val = line.split('=')[1].rstrip().lstrip()[1:-1].split(',')
                    val1 = val[0]
                    val2 = val[1].lstrip().rstrip()
                    i94port_cd.append(key)
                    i94port_state.append(val2)
                    i94port_city.append(val1)
                    
                except IndexError:
                    key = line.split('=')[0].rstrip().lstrip()[1:-1]
                    val = line.split('=')[1].rstrip().lstrip()[1:-1].rstrip()
                    i94port_cd.append(key)
                    i94port_state.append('NaN')
                    i94port_city.append(val)

In [40]:
#reate Pandas DataFrame from arrays and display sample records
i94port_data = {"Port_Code":i94port_cd,"State":i94port_state,"City":i94port_city}
df_i94ports_pd = pd.DataFrame(i94port_data)
df_i94ports_pd.head(5)

,Port_Code,State,City
0,ALC,AK,ALCAN
1,ANC,AK,ANCHORAGE
2,BAR,AK,BAKER AAF - BAKER ISLAND
3,DAC,AK,DALTONS CACHE
4,PIZ,AK,DEW STATION PT LAY DEW


In [41]:
#Display counts
df_i94ports_pd.count()

Port_Code    588
State        588
City         588
dtype: int64

In [39]:
#Counting the bad State records
df_i94ports_pd.query('State == "NaN"')

,Port_Code,State,City
28,MAP,NaN,MARIPOSA AZ
76,WAS,NaN,WASHINGTON DC
572,ZZZ,NaN,MEXICO Land (Banco de Mexico)
573,CHN,NaN,No PORT Code (CHN)
575,MAA,NaN,Abu Dhabi


In [46]:
#Verifying the valid record
df_i94ports_pd.query('Port_Code == "LOS"')

,Port_Code,State,City
44,LOS,CA,LOS ANGELES


In [90]:
#Reaading i94City file and saving in a DataFrame
df_i94city_pd = pd.read_csv("immigration_i94cit_valid.txt",sep="=", names=["Code","Country"], header=None)

In [91]:
df_i94city_pd.head(5)

,Code,Country
0,582,"'MEXICO Air Sea, and Not Reported (I-94, no ..."
1,236,'AFGHANISTAN'
2,101,'ALBANIA'
3,316,'ALGERIA'
4,102,'ANDORRA'


In [92]:
#Removing quotes around the Country
df_i94city_pd["Country"] = df_i94city_pd["Country"].apply(lambda x: x.lstrip().rstrip()[1:-1])
df_i94city_pd.dtypes

Code        int64
Country    object
dtype: object

In [93]:
df_i94city_pd.head(5)

,Code,Country
0,582,"MEXICO Air Sea, and Not Reported (I-94, no lan..."
1,236,AFGHANISTAN
2,101,ALBANIA
3,316,ALGERIA
4,102,ANDORRA


In [13]:
# Create dictionary of valid i94addr codes
i94addr_valid = {}
with open('immigration_i94addr_valid.txt') as f:
     for line in f:
            line = line.rstrip().lstrip()
            if line:
                try:
                    key = line.split('=')[0].rstrip().lstrip()[1:-1]
                    val = line.split('=')[1].rstrip().lstrip()[1:-1]
                    #print(key,val)
                    i94addr_valid[key] = val
                except IndexError:
                    print("Error:",line)
                    raise

In [96]:
df_i94addr_pd = pd.read_csv("immigration_i94addr_valid.txt", sep="=",names =["State_Code","State"],header=None)
df_i94addr_pd.head(5)

,State_Code,State
0,'AL','ALABAMA'
1,'AK','ALASKA'
2,'AZ','ARIZONA'
3,'AR','ARKANSAS'
4,'CA','CALIFORNIA'


In [99]:
#Cleaned single quotes around the data
df_i94addr_pd["State_Code"] = df_i94addr_pd["State_Code"].apply(lambda x : x.lstrip().rstrip()[1:-1])
df_i94addr_pd["State"] = df_i94addr_pd["State"].apply(lambda x : x.lstrip().rstrip()[1:-1])
df_i94addr_pd.head(5)

,State_Code,State
0,AL,ALABAMA
1,AK,ALASKA
2,AZ,ARIZONA
3,AR,ARKANSAS
4,CA,CALIFORNIA


In [100]:
df_i94addr_pd.dtypes

State_Code    object
State         object
dtype: object

In [109]:
#reading and clening VISA DIM data
df_i94visa_pd = pd.read_csv("immigration_i94visa_valid.txt", sep=",",names =["VISA_Code","VISA_Desc"],header=None)
df_i94visa_pd.head()

,VISA_Code,VISA_Desc
0,1,'Business'
1,2,'Pleasure'
2,3,'Student'


In [110]:
#Cleaning single quotes
df_i94visa_pd["VISA_Desc"] = df_i94visa_pd["VISA_Desc"].apply(lambda x : x.lstrip().rstrip()[1:-1])
df_i94visa_pd.head()

,VISA_Code,VISA_Desc
0,1,Business
1,2,Pleasure
2,3,Student


In [10]:
#reading and clening VISA mode data
df_i94mode_pd = pd.read_csv("immigration_i94mode_valid.txt", sep=",",names =["Entry_Code","Entry_Desc"],header=None)
df_i94mode_pd.head()

,Entry_Code,Entry_Desc
0,1,'Air'
1,2,'Sea'
2,3,'Land'
3,9,'Not reported'


In [11]:
#Cleaning single quotes
df_i94mode_pd["Entry_Desc"] = df_i94mode_pd["Entry_Desc"].apply(lambda x : x.lstrip().rstrip()[1:-1])
df_i94mode_pd.head()

,Entry_Code,Entry_Desc
0,1,Air
1,2,Sea
2,3,Land
3,9,Not reported


In [12]:
df_i94mode_pd.dtypes

Entry_Code     int64
Entry_Desc    object
dtype: object

In [113]:
#Reading Global Weather Data
df_global_temp = pd.read_csv('../../data2/GlobalLandTemperaturesByCity.csv',header=0)
df_global_temp.dtypes

dt                                object
AverageTemperature               float64
AverageTemperatureUncertainty    float64
City                              object
Country                           object
Latitude                          object
Longitude                         object
dtype: object

In [118]:
#Finding Los Angeles record
df_global_temp.query('City == "Los Angeles" and Country == "United States"').head(2)

,dt,AverageTemperature,AverageTemperatureUncertainty,City,Country,Latitude,Longitude
4356748,1849-01-01,8.819,2.558,Los Angeles,United States,34.56N,118.70W
4356749,1849-02-01,9.577,1.970,Los Angeles,United States,34.56N,118.70W


In [127]:
#Verifying Date format
df_global_temp.dt.apply(lambda x : x.find("/")).head(1)

0   -1
Name: dt, dtype: int64

In [131]:
#Converting date from string to date data type
df_global_temp.astype({"dt":"datetime64"}).dtypes

dt                               datetime64[ns]
AverageTemperature                      float64
AverageTemperatureUncertainty           float64
City                                     object
Country                                  object
Latitude                                 object
Longitude                                object
dtype: object

In [132]:
df_global_temp.head(3)

,dt,AverageTemperature,AverageTemperatureUncertainty,City,Country,Latitude,Longitude
0,1743-11-01,6.068,1.737,Århus,Denmark,57.05N,10.33E
1,1743-12-01,NaN,NaN,Århus,Denmark,57.05N,10.33E
2,1744-01-01,NaN,NaN,Århus,Denmark,57.05N,10.33E


In [143]:
df_global_temp.count()

dt                               8599212
AverageTemperature               8235082
AverageTemperatureUncertainty    8235082
City                             8599212
Country                          8599212
Latitude                         8599212
Longitude                        8599212
dtype: int64

In [13]:
#Reading US demographics data and printing data types
with open('us-cities-demographics.json','r') as f:
    data = json.load(f)
df_us_info = pd.io.json.json_normalize(data)
df_us_info.dtypes

datasetid                         object
fields.average_household_size    float64
fields.city                       object
fields.count                       int64
fields.female_population         float64
fields.foreign_born              float64
fields.male_population           float64
fields.median_age                float64
fields.number_of_veterans        float64
fields.race                       object
fields.state                      object
fields.state_code                 object
fields.total_population            int64
record_timestamp                  object
recordid                          object
dtype: object

In [14]:
#Sample Data
df_us_info.head(3)

,datasetid,fields.average_household_size,fields.city,fields.count,fields.female_population,fields.foreign_born,fields.male_population,fields.median_age,fields.number_of_veterans,fields.race,fields.state,fields.state_code,fields.total_population,record_timestamp,recordid
0,us-cities-demographics,2.73,Newark,76402,143873.0,86253.0,138040.0,34.6,5829.0,White,New Jersey,NJ,281913,1969-12-31T19:00:00-05:00,85458783ecf5da6572ee00e7120f68eff4fd0d61
1,us-cities-demographics,2.40,Peoria,1343,62432.0,7517.0,56229.0,33.1,6634.0,American Indian and Alaska Native,Illinois,IL,118661,1969-12-31T19:00:00-05:00,a5ad84bdb4d72688fb6ae19a8bee43bcb01f9fea
2,us-cities-demographics,2.77,O'Fallon,2583,43270.0,3269.0,41762.0,36.0,5783.0,Hispanic or Latino,Missouri,MO,85032,1969-12-31T19:00:00-05:00,c54cd5021a16eb5f7b83987742bd495229b2155e


In [15]:
#Renaming the columns
df_us_info.set_axis(['datasetid','average_household_size','city','count','female_population',
                     'foreign_born','male_population','median_age','number_of_veterans','race',
                     'state','state_code','total_population','record_timestamp','recordid'], 
                    axis=1, inplace=True)

In [23]:
#Converting to datetime format and querying sample data
df_us_info.astype({"record_timestamp":"datetime64[ns]"}).query('state_code=="CA" and city == "Los Angeles"')

,datasetid,average_household_size,city,count,female_population,foreign_born,male_population,median_age,number_of_veterans,race,state,state_code,total_population,record_timestamp,recordid
97,us-cities-demographics,2.86,Los Angeles,2177650,2012898.0,1485425.0,1958998.0,35.0,85417.0,White,California,CA,3971896,1970-01-01,7da42fda61238faccac3d43954a8f621a3a51194
554,us-cities-demographics,2.86,Los Angeles,512999,2012898.0,1485425.0,1958998.0,35.0,85417.0,Asian,California,CA,3971896,1970-01-01,e23be85ef2bf6caecf2309ba6dedc868929d1377
729,us-cities-demographics,2.86,Los Angeles,404868,2012898.0,1485425.0,1958998.0,35.0,85417.0,Black or African-American,California,CA,3971896,1970-01-01,cda8c0b63e4c14d174940e1df5d50d2d2491ccfa
1225,us-cities-demographics,2.86,Los Angeles,63758,2012898.0,1485425.0,1958998.0,35.0,85417.0,American Indian and Alaska Native,California,CA,3971896,1970-01-01,f45da4b5c979eb53a8e10a5d4e6ef2a7bb480fbc
1899,us-cities-demographics,2.86,Los Angeles,1936732,2012898.0,1485425.0,1958998.0,35.0,85417.0,Hispanic or Latino,California,CA,3971896,1970-01-01,5212831e25cadd6f383d1bf93274aa17e346adb6


In [22]:
#Verifying Null values
df_us_info.count()

datasetid                 2891
average_household_size    2875
city                      2891
count                     2891
female_population         2888
foreign_born              2878
male_population           2888
median_age                2891
number_of_veterans        2878
race                      2891
state                     2891
state_code                2891
total_population          2891
record_timestamp          2891
recordid                  2891
dtype: int64

In [144]:
#Reading Airport Codes Data
df_airport_cd = pd.read_csv('airport-codes_csv.csv',header=0)

In [145]:
#Checking iso_region format
df_airport_cd[df_airport_cd.iso_region.str.len()>5].head(2)

,ident,type,name,elevation_ft,continent,iso_country,iso_region,municipality,gps_code,iata_code,local_code,coordinates
174,02PR,small_airport,Cuylers Airport,15.0,NaN,PR,PR-U-A,Vega Baja,02PR,NaN,02PR,"-66.36689758300781, 18.45330047607422"
223,03N,small_airport,Utirik Airport,4.0,OC,MH,MH-UTI,Utirik Island,K03N,UTK,03N,"169.852005, 11.222"


In [146]:
df_airport_cd['city'] = df_airport_cd['iso_region'].apply(lambda x: x[x.find('-')+1:])

In [148]:
#Checking iso_region format after cleaning
df_airport_cd[df_airport_cd.iso_region.str.len()>5].head(2)

,ident,type,name,elevation_ft,continent,iso_country,iso_region,municipality,gps_code,iata_code,local_code,coordinates,city
174,02PR,small_airport,Cuylers Airport,15.0,NaN,PR,PR-U-A,Vega Baja,02PR,NaN,02PR,"-66.36689758300781, 18.45330047607422",U-A
223,03N,small_airport,Utirik Airport,4.0,OC,MH,MH-UTI,Utirik Island,K03N,UTK,03N,"169.852005, 11.222",UTI


In [149]:
df_airport_cd.query('iso_country == "US" and municipality=="Los Angeles"').head(2)

,ident,type,name,elevation_ft,continent,iso_country,iso_region,municipality,gps_code,iata_code,local_code,coordinates,city
71,01CN,heliport,Los Angeles County Sheriff's Department Heliport,300.0,NaN,US,US-CA,Los Angeles,01CN,NaN,01CN,"-118.15399932861328, 34.03779983520508",CA
639,0CA0,closed,Drew Medical Center Heliport,180.0,NaN,US,US-CA,Los Angeles,NaN,NaN,NaN,"-118.241997, 33.923302",CA


In [150]:
df_airport_cd.count()

ident           55075
type            55075
name            55075
elevation_ft    48069
continent       27356
iso_country     54828
iso_region      55075
municipality    49399
gps_code        41030
iata_code        9189
local_code      28686
coordinates     55075
city            55075
dtype: int64

In [3]:
#Creating Spark session with sas data
spark = SparkSession.builder.\
config("spark.jars.packages","saurfang:spark-sas7bdat:2.0.0-s_2.11")\
.enableHiveSupport().getOrCreate()

In [183]:
%%time
#Reading sample SAS immigration data set into DataFrame
df_immigration_spark = spark.read.format('com.github.saurfang.sas.spark').load('../../data/18-83510-I94-Data-2016/i94_apr16_sub.sas7bdat')

CPU times: user 1.29 ms, sys: 239 µs, total: 1.53 ms
Wall time: 27.7 ms


In [154]:
%%time
#Total records
df_immigration_spark.count()

3096313

In [162]:
%%time
#Checking for the duplicate records and matching with total records
df_immigration_spark.distinct().count()

CPU times: user 8.57 ms, sys: 861 µs, total: 9.43 ms
Wall time: 53.1 s


3096313

In [5]:
%%time
#finding the PK column
df_immigration_spark.select('admnum').distinct().count()

CPU times: user 11.1 ms, sys: 609 µs, total: 11.7 ms
Wall time: 55.2 s


3075579

In [184]:
#sample data
df_immigration_spark.show(5)

+-----+------+------+------+------+-------+-------+-------+-------+-------+------+-------+-----+--------+--------+-----+-------+-------+-------+-------+-------+--------+------+------+-------+--------------+-----+--------+
|cicid| i94yr|i94mon|i94cit|i94res|i94port|arrdate|i94mode|i94addr|depdate|i94bir|i94visa|count|dtadfile|visapost|occup|entdepa|entdepd|entdepu|matflag|biryear| dtaddto|gender|insnum|airline|        admnum|fltno|visatype|
+-----+------+------+------+------+-------+-------+-------+-------+-------+------+-------+-----+--------+--------+-----+-------+-------+-------+-------+-------+--------+------+------+-------+--------------+-----+--------+
|  6.0|2016.0|   4.0| 692.0| 692.0|    XXX|20573.0|   null|   null|   null|  37.0|    2.0|  1.0|    null|    null| null|      T|   null|      U|   null| 1979.0|10282016|  null|  null|   null| 1.897628485E9| null|      B2|
|  7.0|2016.0|   4.0| 254.0| 276.0|    ATL|20551.0|    1.0|     AL|   null|  25.0|    3.0|  1.0|20130811|     SE

In [4]:
#user defined functions to convert the dates and remove * from data
def convert_datetime(x):
    try:
        start = datetime(1960, 1, 1)
        return start + timedelta(days=int(x))
    except:
        return datetime(1900,1,1) 
udf_datetime_from_sas = udf(lambda x: convert_datetime(x), T.DateType())

def convert_YYYYMMDD_datetime(x):
    try:
        x = str(x)
        return datetime(int(x[0:4]),int(x[4:6]),int(x[6:8]))

    except:
        return datetime(1900,1,1) 
udf_datetime_from_YYYYMMDD = udf(lambda x: convert_YYYYMMDD_datetime(x), T.DateType())

def convert_MMDDYYYY_datetime(x):
    try:
        x = str(x)
        return datetime(int(x[4:8]),int(x[0:2]),int(x[2:4]))
    except:
        return datetime(1900,1,1) 
udf_datetime_from_MMDDYYYY = udf(lambda x: convert_MMDDYYYY_datetime(x), T.DateType())

def remove_Special_Char(x):
    try:
        return x.lstrip().rstrip().lstrip('*').rstrip('*')
    except:
        return '-'
    
udf_remove_Special_Char = udf(lambda x: remove_Special_Char(x),T.StringType())


In [154]:
#Calling udf to convert arrival date
df_immigration_spark = df_immigration_spark.withColumn("Arrival_Date",udf_datetime_from_sas(df_immigration_spark.arrdate))

In [116]:
df_immigration_spark.where(df_immigration_spark.Arrival_Date.isNull()).show(5)

+-----+-----+------+------+------+-------+-------+-------+-------+-------+------+-------+-----+--------+--------+-----+-------+-------+-------+-------+-------+-------+------+------+-------+------+-----+--------+------------+
|cicid|i94yr|i94mon|i94cit|i94res|i94port|arrdate|i94mode|i94addr|depdate|i94bir|i94visa|count|dtadfile|visapost|occup|entdepa|entdepd|entdepu|matflag|biryear|dtaddto|gender|insnum|airline|admnum|fltno|visatype|Arrival_Date|
+-----+-----+------+------+------+-------+-------+-------+-------+-------+------+-------+-----+--------+--------+-----+-------+-------+-------+-------+-------+-------+------+------+-------+------+-----+--------+------------+
+-----+-----+------+------+------+-------+-------+-------+-------+-------+------+-------+-----+--------+--------+-----+-------+-------+-------+-------+-------+-------+------+------+-------+------+-----+--------+------------+



In [158]:
df_immigration_spark = df_immigration_spark.withColumn("I94_added_date",udf_datetime_from_YYYYMMDD(df_immigration_spark.dtadfile))\
.withColumn("VISA_Expiry_date",udf_datetime_from_MMDDYYYY(df_immigration_spark.dtaddto))

In [159]:
df_immigration_spark.show(5)

+-----+------+------+------+------+-------+-------+-------+-------+-------+------+-------+-----+--------+--------+-----+-------+-------+-------+-------+-------+--------+------+------+-------+--------------+-----+--------+------------+--------------+----------------+
|cicid| i94yr|i94mon|i94cit|i94res|i94port|arrdate|i94mode|i94addr|depdate|i94bir|i94visa|count|dtadfile|visapost|occup|entdepa|entdepd|entdepu|matflag|biryear| dtaddto|gender|insnum|airline|        admnum|fltno|visatype|Arrival_Date|I94_added_date|VISA_Expiry_date|
+-----+------+------+------+------+-------+-------+-------+-------+-------+------+-------+-----+--------+--------+-----+-------+-------+-------+-------+-------+--------+------+------+-------+--------------+-----+--------+------------+--------------+----------------+
|  6.0|2016.0|   4.0| 692.0| 692.0|    XXX|20573.0|   null|   null|   null|  37.0|    2.0|  1.0|    null|    null| null|      T|   null|      U|   null| 1979.0|10282016|  null|  null|   null| 1.89762

In [186]:
#Converting all decimals to integer
df_immigration_spark = df_immigration_spark.withColumn("i94yr",df_immigration_spark.i94yr.cast('int'))\
.withColumn("i94mon",df_immigration_spark.i94mon.cast('int'))\
.withColumn("i94cit",df_immigration_spark.i94cit.cast('int'))\
.withColumn("i94res",df_immigration_spark.i94res.cast('int'))\
.withColumn("arrdate",df_immigration_spark.arrdate.cast('int'))\
.withColumn("i94mode",df_immigration_spark.i94mode.cast('int'))\
.withColumn("depdate",df_immigration_spark.depdate.cast('int'))\
.withColumn("i94bir",df_immigration_spark.i94bir.cast('int'))\
.withColumn("dtadfile",df_immigration_spark.dtadfile.cast('int'))\
.withColumn("biryear",df_immigration_spark.biryear.cast('int'))\
.withColumn("admnum",df_immigration_spark.admnum.cast('int'))\
.withColumn("i94visa",df_immigration_spark.i94visa.cast('int'))


In [187]:
#Cleaning other fields
df_immigration_spark = df_immigration_spark.withColumn("arrdate",udf_datetime_from_sas(df_immigration_spark.arrdate))\
.withColumn("depdate",udf_datetime_from_sas(df_immigration_spark.depdate))\
.withColumn("dtadfile",udf_datetime_from_YYYYMMDD(df_immigration_spark.dtadfile))\
.withColumn("dtaddto",udf_datetime_from_MMDDYYYY(df_immigration_spark.dtaddto))\
.withColumn("airline",udf_remove_Special_Char(df_immigration_spark.airline))

In [122]:
%%time
#Top 4 records where cicid=610
df_immigration_spark.filter(df_immigration_spark.cicid==610).show(4)

+-----+-----+------+------+------+-------+----------+-------+-------+----------+------+-------+-----+----------+--------+-----+-------+-------+-------+-------+-------+----------+------+------+-------+----------+-----+--------+
|cicid|i94yr|i94mon|i94cit|i94res|i94port|   arrdate|i94mode|i94addr|   depdate|i94bir|i94visa|count|  dtadfile|visapost|occup|entdepa|entdepd|entdepu|matflag|biryear|   dtaddto|gender|insnum|airline|    admnum|fltno|visatype|
+-----+-----+------+------+------+-------+----------+-------+-------+----------+------+-------+-----+----------+--------+-----+-------+-------+-------+-------+-------+----------+------+------+-------+----------+-----+--------+
|610.0| 2016|     4|   103|   103|    LVG|2016-04-01|      1|     NV|2016-04-02|    58|      2|  1.0|2016-04-01|     BRL| null|      G|      R|   null|      M|   1958|2016-09-30|     M|  null|     GA|2147483647|VPCSW|      B2|
+-----+-----+------+------+------+-------+----------+-------+-------+----------+------+-----

In [155]:
#Reading SAS source file names
src_sas_path='../../data/18-83510-I94-Data-2016'
sas_source_files =[]
for i in os.listdir(src_sas_path):
    sas_source_files.append(os.path.join(src_sas_path,i))

In [156]:
#Printing the source file names
for i in sas_source_files:
    print(i)

../../data/18-83510-I94-Data-2016/i94_apr16_sub.sas7bdat
../../data/18-83510-I94-Data-2016/i94_sep16_sub.sas7bdat
../../data/18-83510-I94-Data-2016/i94_nov16_sub.sas7bdat
../../data/18-83510-I94-Data-2016/i94_mar16_sub.sas7bdat
../../data/18-83510-I94-Data-2016/i94_jun16_sub.sas7bdat
../../data/18-83510-I94-Data-2016/i94_aug16_sub.sas7bdat
../../data/18-83510-I94-Data-2016/i94_may16_sub.sas7bdat
../../data/18-83510-I94-Data-2016/i94_jan16_sub.sas7bdat
../../data/18-83510-I94-Data-2016/i94_oct16_sub.sas7bdat
../../data/18-83510-I94-Data-2016/i94_jul16_sub.sas7bdat
../../data/18-83510-I94-Data-2016/i94_feb16_sub.sas7bdat
../../data/18-83510-I94-Data-2016/i94_dec16_sub.sas7bdat


In [172]:
x = str(sas_source_files[0])

In [177]:
#Checking to extract the month
x[x.find('/i94_')+5:x.find('_sub')]

'apr16'

In [204]:
%%time
#1. Converting all non date fields to human readable date format.
#2. Convert float data columns trypes to int.
#3. Remove * infront of Airline code.
#4. When reading other Text static files into dataframe, clean the signle quotes.

def read_sas_files(file_path):
    file_path = str(file_path)
    tgt_path = file_path[file_path.find('/i94_')+5:file_path.find('_sub')]
    
    df_spark = spark.read.format('com.github.saurfang.sas.spark').load(file_path)
    df_spark = df_spark.withColumn("i94yr",df_spark.i94yr.cast('int'))\
      .withColumn("i94mon",df_spark.i94mon.cast('int'))\
      .withColumn("i94cit",df_spark.i94cit.cast('int'))\
      .withColumn("i94res",df_spark.i94res.cast('int'))\
      .withColumn("arrdate",df_spark.arrdate.cast('int'))\
      .withColumn("i94mode",df_spark.i94mode.cast('int'))\
      .withColumn("depdate",df_spark.depdate.cast('int'))\
      .withColumn("i94bir",df_spark.i94bir.cast('int'))\
      .withColumn("dtadfile",df_spark.dtadfile.cast('int'))\
      .withColumn("biryear",df_spark.biryear.cast('int'))\
      .withColumn("admnum",df_spark.admnum.cast('int'))\
      .withColumn("i94visa",df_spark.i94visa.cast('int'))
    
    df_spark = df_spark.withColumn("arrdate",udf_datetime_from_sas(df_spark.arrdate))\
      .withColumn("depdate",udf_datetime_from_sas(df_spark.depdate))\
      .withColumn("dtadfile",udf_datetime_from_YYYYMMDD(df_spark.dtadfile))\
      .withColumn("dtaddto",udf_datetime_from_MMDDYYYY(df_spark.dtaddto))\
      .withColumn("airline",udf_remove_Special_Char(df_spark.airline))
    #df_spark = df_spark.limit(10)
    df_spark.write.parquet("sas_data1/"+tgt_path)
    return [file_path,df_spark]

#using Concurrent futures to execute the reading and writing the files in Parallel.
#This work only if the server has CPU and Memory capabilities to process the tasks in parallel.

max_workers = len(sas_source_files)
with ThreadPoolExecutor(max_workers=max_workers) as executor:
    future_count = {executor.submit(read_sas_files,file_path) : file_path for file_path in sas_source_files}
    for record in concurrent.futures.as_completed(future_count):
        print(record.result()[0])
        #print(record.result()[1].show(2))

../../data/18-83510-I94-Data-2016/i94_apr16_sub.sas7bdat
../../data/18-83510-I94-Data-2016/i94_mar16_sub.sas7bdat
../../data/18-83510-I94-Data-2016/i94_dec16_sub.sas7bdat
../../data/18-83510-I94-Data-2016/i94_nov16_sub.sas7bdat
../../data/18-83510-I94-Data-2016/i94_feb16_sub.sas7bdat
../../data/18-83510-I94-Data-2016/i94_jul16_sub.sas7bdat
../../data/18-83510-I94-Data-2016/i94_sep16_sub.sas7bdat
../../data/18-83510-I94-Data-2016/i94_aug16_sub.sas7bdat
../../data/18-83510-I94-Data-2016/i94_oct16_sub.sas7bdat
../../data/18-83510-I94-Data-2016/i94_jan16_sub.sas7bdat
../../data/18-83510-I94-Data-2016/i94_may16_sub.sas7bdat
../../data/18-83510-I94-Data-2016/i94_jun16_sub.sas7bdat
CPU times: user 258 ms, sys: 83 ms, total: 341 ms
Wall time: 26min 14s


In [5]:
#Reading all Parquet files
df_spark_par=spark.read.parquet("sas_data1/*/*")

In [211]:
%%time
#Grouping By VISATYPE=B2
df_spark_par.where(df_spark_par.visatype=="B2").groupBy(df_spark_par.i94mon).count().collect()

CPU times: user 6.03 ms, sys: 330 µs, total: 6.36 ms
Wall time: 8.52 s


[Row(i94mon=12, count=1544994),
 Row(i94mon=1, count=942869),
 Row(i94mon=6, count=1463607),
 Row(i94mon=3, count=1085794),
 Row(i94mon=5, count=1372589),
 Row(i94mon=9, count=1327985),
 Row(i94mon=4, count=1117897),
 Row(i94mon=8, count=1333639),
 Row(i94mon=7, count=1767133),
 Row(i94mon=10, count=1251412),
 Row(i94mon=11, count=1081584),
 Row(i94mon=2, count=899282)]

In [8]:
%%time
#Grouping By VISATYPE=B2 and sort in descending order
df_spark_par.where(df_spark_par.visatype=="B2").groupBy(df_spark_par.i94mon).count().sort("count",ascending=False).collect()

CPU times: user 12.5 ms, sys: 739 µs, total: 13.2 ms
Wall time: 19.2 s


[Row(i94mon=7, count=1767133),
 Row(i94mon=12, count=1544994),
 Row(i94mon=6, count=1463607),
 Row(i94mon=5, count=1372589),
 Row(i94mon=8, count=1333639),
 Row(i94mon=9, count=1327985),
 Row(i94mon=10, count=1251412),
 Row(i94mon=4, count=1117897),
 Row(i94mon=3, count=1085794),
 Row(i94mon=11, count=1081584),
 Row(i94mon=1, count=942869),
 Row(i94mon=2, count=899282)]

### Step 3: Define the Data Model
#### 3.1 Conceptual Data Model
Based on the Source data, prepared the below conceptual model and you can derive many solutions based on this data

<img src="Immigration and Temperature DB Logical Design.jpg" >


#### 3.2 Mapping Out Data Pipelines

##### Steps:
**General Solution (Local Machine):**
* Extract source files into DataFrame.
* Transform/Cleanup the data in the DataFrame.
* Load all DataFrames into corresponding **Parquet** files.
* Analyze the data.

**Cloud Solution (AWS EMR):**
* Extract source data into DataFrames.
* Create Tables in Cloud Database.
* Transform/Cleanup the data in the DataFrame.
* Load the data into DIM Tables.
* Load the Fact table.
* Analyze the data.



### Step 4: Run Pipelines to Model the Data 
#### 4.1 Create the data model

##### Physical Data Model (Star Schema):

###### Dimention Tables:
<p style = "font-size:12px ; background-color: #f5f5f5">
CREATE TABLE AIRPORTS_CODES_DIM(<br>
IDENTIFICATION VARCHAR(10) NOT NULL,<br>
NAME VARCHAR(100),<br>
TYPE VARCHAR(30),<br>
CONTINENT VARCHAR(10),<br>
COUNTRY VARCHAR(10),<br>
CITY VARCHAR(10),<br>
MUNICIPALITY VARCHAR(50),<br>
PRIMARY KEY(IDENTIFICATION))<br>
DISTSTYLE ALL;<br>
<br>
CREATE TABLE US_CITY_DEMOGRAPHIC_DIM(<br>
ID INT NOT NULL,<br>
STATE VARCHAR(10),<br>
CITY VARCHAR(50),<br>
RECORD_TIMESTAMP TIMESTAMP,<br>
TOTAL_POPULATION  BIGINT,<br>
MALE_POPULATION BIGINT,<br>
FEMALE_POPULATION BIGINT,<br>
COUNT BIGINT,<br>
MEDIAN_AGE  SMALLINT,<br>
AVERAGE_HOUSEHOLD_SIZE SMALLINT,<br>
RACE  VARCHAR(50),<br>
FOREIGN_BORN BIGINT,<br>
NUMBER_OF_VETERANS  SMALLINT,<br>
PRIMARY KEY(ID))<br>
DISTSTYLE ALL;<br>
<br>
CREATE TABLE 194_PORT_DIM(<br>
PORT_CODE VARCHAR(5),<br>
STATE VARCHAR(5),<br>
CITY VARCHAR(30),<br>
PRIMARY KEY(PORT_CODE))<br>
DISTSTYLE ALL;<br>
<br>
CREATE TABLE 194_ENTRY_MODE_DIM(<br>
ENTRY_CODE SMALLINT NOT NULL,<br>
ENTRY_DESC VARCHAR(15) NOT NULL,<br>
PRIMARY KEY(ENTRY_CODE))<br>
DISTSTYLE ALL;<br>
<br>
CREATE TABLE 194_ADDRESS_DIM(<br>
STATE_CODE  CHAR(2) NOT NULL,<br>
STATE VARCHAR(30),<br>
PRIMARY KEY(STATE_CODE))<br>
DISTSTYLE ALL;<br>
<br>
CREATE TABLE 194_VISA_DIM(<br>
VISA_CODE SMALLINT NOT NULL,<br>
VISA_DESC VARCHAR(20) NOT NULL,<br>
PRIMARY KEY(VISA_CODE))<br>
DISTSTYLE ALL;<br>
<br>
CREATE TABLE DATE_DIM(<br>
DATE DATE NOT NULL,<br>
DAY  VARCHAR(50),<br>
MONTH VARCHAR(50),<br>
YEAR INTEGER<br>
DAY_OF_WEEK VARCHAR(50),<br>
DAY_OF_MONTH VARCHAR(50),<br>
WEEK_OF_YEAR VARCHAR(50),<br>
HOUR TIMESTAMP,<br>
MINUTES TIMESTAMP,<br>
PRIMARY KEY(DATE))<br>
DISTSTYLE ALL;<br>
<br>
CREATE TABLE GLOBAL_WEATHER_DIM(<br>
ID SMALLINT NOT NULL,<br>
DATE DATE,<br>
CITY VARCHAR(70),<br>
COUNTRY VARCHAR(30),<br>
AVERAGE_TEMPERATURE SMALLINT,<br>
AVERAGE_TEMPERATURE_UNCERTAIN SMALLINT,<br>
LATITUDE SMALLINT,<br>
LONGITUDE SMALLINT,<br>
PRIMARY KEY(ID))<br>
DISTSTYLE ALL;<br>
<br>
CREATE TABLE 194_CIT_RES_DIM(<br>
CODE SMALLINT NOT NULL,<br>
COUNTRY VARCHAR(50),<br>
PRIMARY KEY(CODE))<br>
DISTSTYLE ALL;</p>
<br>

###### Fact Table:
<p style = "font-size:12px ; background-color: #f5f5f5">
CREATE TABLE IF NOT EXISTS 194_IMMIGRATION_FACT(<br>
ADMISSION_NUMBER INTEGER NOT NULL,<br>
YEAR YEAR NOT NULL,<br>
MONTH MONTH NOT NULL,<br>
I94_CITY  SMALLINT NOT NULL,<br>
I94_RES  SMALLINT NOT NULL,<br>
PORT  VARCHAR(10),<br>
ARRIVAL_DATE DATE NOT NULL,<br>
ENTRY_MODE SMALLINT,<br>
ARRIVAL_STATE VARCHAR(5),<br>
AGE SMALLINT,<br>
VISA_CODE SMALLINT,<br>
VISA_POST VARCHAR(20),<br>
OCCUPATION VARCHAR(50),<br>
ARRIVAL_FLAG CHAR(1),<br>
DEPARTURE_FLAG CHAR(1),<br>
UPDATE_FLAG CHAR(1),<br>
MATCH_FLAG CHAR(1),<br>
VISA_EXPIRE_DATE DATE NOT NULL,<br>
GENDER CHAR(1),<br>
AIRLINE VARCHAR(5),<br>
FLIGHT_NUMBER VARCHAR(10),<br>
VISA_TYPE VARCHAR(10),<br>
PRIMARY KEY(ADMISSION_NUMBER))<br>
);<br>
<br>
</p>

In [9]:
#Reading SAS source file names
src_sas_path='../../data/18-83510-I94-Data-2016'
sas_source_files =[]
for i in os.listdir(src_sas_path):
    sas_source_files.append(os.path.join(src_sas_path,i))

In [ ]:
#Printing the source file names
for i in sas_source_files:
    print(i)

In [ ]:
%%time
#1. Converting all non date fields to human readable date format.
#2. Convert float data columns trypes to int.
#3. Remove * infront of Airline code.
#4. When reading other Text static files into dataframe, clean the signle quotes.

def read_sas_files(file_path):
    file_path = str(file_path)
    tgt_path = file_path[file_path.find('/i94_')+5:file_path.find('_sub')]
    
    df_spark = spark.read.format('com.github.saurfang.sas.spark').load(file_path)
    df_spark = df_spark.withColumn("i94yr",df_spark.i94yr.cast('int'))\
      .withColumn("i94mon",df_spark.i94mon.cast('int'))\
      .withColumn("i94cit",df_spark.i94cit.cast('int'))\
      .withColumn("i94res",df_spark.i94res.cast('int'))\
      .withColumn("arrdate",df_spark.arrdate.cast('int'))\
      .withColumn("i94mode",df_spark.i94mode.cast('int'))\
      .withColumn("depdate",df_spark.depdate.cast('int'))\
      .withColumn("i94bir",df_spark.i94bir.cast('int'))\
      .withColumn("dtadfile",df_spark.dtadfile.cast('int'))\
      .withColumn("biryear",df_spark.biryear.cast('int'))\
      .withColumn("admnum",df_spark.admnum.cast('int'))\
      .withColumn("i94visa",df_spark.i94visa.cast('int'))
    
    df_spark = df_spark.withColumn("arrdate",udf_datetime_from_sas(df_spark.arrdate))\
      .withColumn("depdate",udf_datetime_from_sas(df_spark.depdate))\
      .withColumn("dtadfile",udf_datetime_from_YYYYMMDD(df_spark.dtadfile))\
      .withColumn("dtaddto",udf_datetime_from_MMDDYYYY(df_spark.dtaddto))\
      .withColumn("airline",udf_remove_Special_Char(df_spark.airline))
    #df_spark = df_spark.limit(10)
    df_spark.write.parquet("sas_data1/"+tgt_path)
    return [file_path,df_spark]

#using Concurrent futures to execute the reading and writing the files in Parallel.
#This work only if the server has CPU and Memory capabilities to process the tasks in parallel.

max_workers = len(sas_source_files)
with ThreadPoolExecutor(max_workers=max_workers) as executor:
    future_count = {executor.submit(read_sas_files,file_path) : file_path for file_path in sas_source_files}
    for record in concurrent.futures.as_completed(future_count):
        print('File '+record.result()[0]+' process completed..')
        #print(record.result()[1].show(2))

#### 4.2 Data Quality Checks
Explain the data quality checks you'll perform to ensure the pipeline ran as expected. These could include:
 * Integrity constraints on the relational database (e.g., unique key, data type, etc.)
 * Unit tests for the scripts to ensure they are doing the right thing
 * Source/Count checks to ensure completeness
 
Run Quality Checks

In [ ]:
#Reading all Parquet files
df_spark_par=spark.read.parquet("sas_data1/*/*")

In [13]:
%%time
#distinct records
df_spark_par.count()

CPU times: user 1.31 ms, sys: 0 ns, total: 1.31 ms
Wall time: 672 ms


40790529

In [12]:
%%time
#distinct records
df_spark_par.distinct().count()

CPU times: user 50.3 ms, sys: 8.97 ms, total: 59.2 ms
Wall time: 6min 15s


40790529

In [ ]:
%%time
#Grouping By VISATYPE=B2 and sort in descending order
df_spark_par.where(df_spark_par.visatype=="B2").groupBy(df_spark_par.i94mon).count().sort("count",ascending=False).collect()

#### 4.3 Data dictionary 
Create a data dictionary for your data model. For each field, provide a brief description of what the data is and where it came from. You can include the data dictionary in the notebook or in a separate file.

#### Step 5: Complete Project Write Up
* Clearly state the rationale for the choice of tools and technologies for the project.
* Propose how often the data should be updated and why.
* Write a description of how you would approach the problem differently under the following scenarios:
 * The data was increased by 100x.
 * The data populates a dashboard that must be updated on a daily basis by 7am every day.
 * The database needed to be accessed by 100+ people.